In [ ]:
from jinjasql import JinjaSql

import sqlparse
import sqlalchemy
import json

import pandas as pd

In [ ]:
def load_json(source_file):
    with open(source_file, 'r') as f:
        json_data = json.load(f)
        
    return json_data

def connect_mysql(source_file):
    credential = load_json(source_file)
    
    username = credential['username']
    password = credential['password']
    host = credential['host']
    port = credential['port']
    db = credential['database']

    engine = sqlalchemy.create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{db}')
    return engine

# Service

In [ ]:
class BaseSvc:
    def __init__(self, credential_file):
        self.engine = connect_mysql(credential_file)
        
        self.base_template = None
        with open ('../resources/template/base_template.txt') as file:
            self.base_template = file.read()
        
    @staticmethod
    def generate_query(template, params, beutify=True):
        query, bind_params = JinjaSql(param_style='pyformat').prepare_query(template, params)
        query = query % bind_params

        if beutify:
            query = sqlparse.format(query, reindent=True, keyword_case='upper')
        return query
        
    def execute(self, table, select='*',
                join={}, where=[], group_by=[], order_by={},
                limit=None, offset=None, chunksize=10_000, show_query=False, run_query=True):
        
        return self.exec_template(self.base_template, table, select=select,
                                  join=join, where=where, group_by=group_by, order_by=order_by,
                                  limit=limit, offset=offset, chunksize=chunksize,
                                  show_query=show_query, run_query=run_query)
    
    def exec_template(self, template, table, select='*',
                      join={}, where=[], group_by=[], order_by={}, custom={},
                      limit=None, offset=None, chunksize=10_000, show_query=False, run_query=True):
        
        '''
        Parameters:
        -----------
        template:
            - SQL query template with Jinja syntax.
            
        table:
            - name of main table.
            - can include alias name, e.g. 'table_name alias'
            
        select:
            - '*' or {} will select all fields
            - specify original fields name and outputs name, e.g.
                {
                    'as_field_1': 'alias.field_1',
                    ...
                }
            
        join:
            - specify tables to join with. e.g.
                {
                    'table_2': {
                        'type': 'LEFT JOIN',
                        'on': 'table_2.key = table_1.key'
                    },
                    ...
                }
            - specify subquery to join with, e.g.
                {
                    't2': {
                        'query': 'SELECT * FROM table_2',
                        'type': 'LEFT JOIN',
                        'on': 't2.key = table_1.key'
                    }
                }
                
        where:
            - specify filtering conditions, e.g.
                [
                    {
                        'column': 'field_name',
                        'operator': '=',
                        'value': "'sometext'"
                    },
                    ...
                ]
                
        group_by:
            - specify aggregation keys, e.g. ['alias.field_1', ...]
            
        order_by:
            - specify sorting fields, e.g.
                {
                    'alias.field_1': 'ASC',
                    ...
                }
                
        custom:
            - specify customized parameters for customized templates
            - ensure template placeholder starts with 'custom' keyword, e.g. {{ custom.args }}
            
        limit:
            - limit number of rows returned in query result.
            
        offset:
            - offset query result.
        '''
        
        query_params = {
            'select': select,
            'from': table,
            'join': join,
            'where': where,
            'group_by': group_by,
            'order_by': order_by,
            'limit': limit,
            'offset': offset,
            'custom': custom
        }
        query = self.generate_query(template, query_params)
        if show_query:
            print(query)
        
        if run_query:
            df_chunks = pd.read_sql_query(query, self.engine, chunksize=chunksize)
            
            try:
                df = pd.concat(df_chunks)
            except ValueError:
                # Select table fields
                df = pd.read_sql_query(f'SELECT * FROM {table} LIMIT 0', self.engine)
            
            # Convert date format
            columns = [x[0] for x in df.dtypes.items() if x[1] == '<M8[ns]']
            for column in columns:
                df[column] = df[column].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
            return df
        
        return query

In [ ]:
class MT4Svc(BaseSvc):
    def __init__(self, server):
        super().__init__('../resources/credential/MySQL.json')
        
        self.schema = self.server_schema(server)
        
    def server_schema(self, server):
        schema = '' if server == 1 else server
        schema = f'reports{schema}'
        return schema
        
    def exec_servers(self, table, servers=[1,2,3,4], select='*',
                     join={}, where=[], group_by=[], order_by={},
                     limit=None, offset=None, chunksize=10_000, show_query=False, run_query=True):
        
        results = []
        is_df = False
        for server in servers:
            schema = self.server_schema(server)
            query_table = f'{schema}.{table}'
            
            result = super().execute(query_table, select=select,
                                     join=join, where=where, group_by=group_by, order_by=order_by,
                                     limit=limit, offset=offset, chunksize=chunksize, show_query=show_query, run_query=run_query)
            
            is_df = 'DataFrame' in str(type(result))
            if is_df:
                result['server'] = server
            else:
                result = result.replace('SELECT', f'SELECT {server} AS `server`, ')
                result = sqlparse.format(result, reindent=True, keyword_case='upper')
            results.append(result)
            
        return pd.concat(results, ignore_index=True) if is_df else ' UNION ALL '.join(results)

# Example

In [ ]:
mt4_svc = MT4Svc(server=1)

table = f'{mt4_svc.schema}.MT4_USERS'
result_df = mt4_svc.execute(table, show_query=True)
result_df.head()

In [ ]:
mt4_svc = MT4Svc(server=2)

table = f'{mt4_svc.schema}.MT4_TRADES'
result_df = mt4_svc.execute(table, limit=100, offset=0, show_query=True)
result_df.head()

In [ ]:
mt4_svc = MT4Svc(server=2)

# Sub query
table = f'{mt4_svc.schema}.MT4_TRADES'
select = {
    'login': 'login',
    'cmd': 'cmd',
    'SUM(profit)': 'total_profit',
}
where = [
    {
        'column': 'cmd',
        'operator': 'IN',
        'value': '(0, 1)'
    }
]
group_by = ['login', 'cmd']
query = mt4_svc.execute(table, select=select, where=where, group_by=group_by, show_query=False, run_query=False)

# Main query
table = f'{mt4_svc.schema}.MT4_USERS u'
select = {
    'u.login': 'login',
    'u.name': 'name',
    't.cmd': 'cmd',
    't.total_profit': 'total_profit'
}
join = {
    't': {
        'query': query,
        'type': 'RIGHT JOIN',
        'on': 't.login = u.login'
    }
}
order_by = {
    't.login': 'ASC'
}
result_df = mt4_svc.execute(table, select=select, join=join, order_by=order_by, show_query=True)
result_df.head()

In [ ]:
mt4_svc = MT4Svc(server=None)

table = f'MT4_USERS'
select = {
    'login': 'login',
    'name': 'name'
}
where = [
    {
        'column': 'login',
        'operator': '>=',
        'value': 90
    },
    {
        'column': 'login',
        'operator': '<=',
        'value': 100
    }
]
result_df = mt4_svc.exec_servers(table, select=select, where=where, show_query=True)
result_df.head()

In [1]:
reducer_maps = {
    'a': ('A1', 'A2'),
    'b': ('B1', 'B2'),
    'c': ('C1', 'C2'),
    'd': ('D1', 'D2'),
    'e': ('E1', 'E2'),
}

In [3]:
{k: v[0] for k,v in reducer_maps.items()}

{'a': 'A1', 'b': 'B1', 'c': 'C1', 'd': 'D1', 'e': 'E1'}

In [4]:
{k: v[1] for k,v in reducer_maps.items()}

{'a': 'A2', 'b': 'B2', 'c': 'C2', 'd': 'D2', 'e': 'E2'}